In [1]:



## Before execution please note that filepath, a test string should be copied to the driver code in the end of this code

import re
from nltk.corpus import stopwords

#first, we will read the file.
#######PLEASE NOTE THAT YOU HAVE TO COPY THE SAME FILE PATH IN THE DRIVER CODE MENTIONED IN THE LAST PART OF THE CODE BEFORE EXECUTING THE PROGRAM#######
def read_file(filename='acronym_example1.txt'): #The same file or filepath needs to be mentioned in the s= read_file() in DRIVER CODE IN THE LAST LINES OF THE CODE. You can easily find the driver code below by reaching the last few lines of the code
    f = open(filename, "r", encoding="utf8")
    s = f.read()
    return s

## QUESTION 1. PART 1 DEFINING THE FUNCTION TO FIND THE ACRONYMS
## BEFORE executing, PLEASE NOTE that if you are testing the code by pasting a string or opening a file,
#  the same string should be copied to the acronyms = find_acronyms(s) in DRIVER CODE below

def find_acronyms(s):
 
    acronyms_nf=[]
    #Using the regex function and defining a pattern for the identification.
    acronyms_nf=re.findall(r'(?:[A-Z]{2,}[:alpha:]|[a-z][A-Z]{2,}|\([A-Z]{2,}\)|[A-Z]{2,}[-A-Z]*)',s)
    acronyms_nf=list(dict.fromkeys(acronyms_nf))
    acronyms=[]
    for acronym in acronyms_nf:
       
     acronym = acronym.replace("(", "").replace(")", "")
     if  acronym not in acronyms:
         acronyms.append(acronym )
    return acronyms
### QUESTION 1 . PART 2 FINDING THE LONG FORMS

## Initially we find the stopwords and remove them
def remove_mystopwords(sentence):
    tokens = str(sentence).split(" ") ##Splitting the sentence and words ##
    tokens_filtered= [word for word in tokens if not word in list(set(stopwords.words("english")))] ##We have applied a filter 
    return (" ".join(tokens_filtered))

def find_long_forms(s, acronyms):
    sentences= s.split('.')
    d={}
    acronym_sentence ={}
    # cycle through sentences and index which sentence the acronym appears for the first time
    for acronym in acronyms:
        for sent in sentences:
            if acronym in sent and acronym not in acronym_sentence.keys():
                sent = sent.replace("'s"," ").replace("(", "").replace(")", "").replace("-", " ").replace("'", " ")
                acronym_sentence[acronym]= sent
    # Searching the acronyms in the sentences and returning the associated long forms           
    for acronym in acronym_sentence:
        sentence = acronym_sentence.get(acronym)
        new_filtered_sentence = remove_mystopwords(sentence) #Preparing a new filtered sentence
        full_form = find_org(new_filtered_sentence, acronym)
        d[acronym] = full_form  
    
    return d
##QUESTION 1 PART 3 REPLACING THE ACRONYM WITH THE FULL FORM.
# In some cases, full form and acronym might be together but the sentence has been regenerated by replacing the acronym in quotation mark or comma, with the full form and associated acronym.
def replace_acronyms(s, d):
    #doc = nlp(s)
    new_s = " "
    for sent in s.split("."):
    #rule 1 check for dictionary keys, if short form and long form both are present then don't replace
        if words_in_string(d.keys(), sent) and not words_in_string(d.values(),sent):
            for word in words_in_string(d.keys(), sent):
                if d[word] and word:
                    new_s = new_s + sent.replace(word, d[word])
                else:
                    new_s = new_s + sent
        else:
            new_s = new_s + sent
    # rule 2 if only short form is there in stenence find it and replace with first letter capital
    return new_s



def find_org(text, acronym):
    # good for abbrevs that are in all caps.
    try:
        text = text.split(acronym)[0]
    except:
        text = text
    orig_text_token_list = str(text).split(" ")
    text_token_list = [x.title() for x in orig_text_token_list]
    stop_w = [x.title() for x in list(set(stopwords.words("english")))]
   
    potential_match = []
    #if acronym is surrounded by parens, remove
    acronym = acronym.replace("(", "").replace(")", "")
    # return all indices of words that start w first letter of acronym
   
    # remove 's and
    for potential_start_idx in [
        i for i, x in enumerate(text_token_list) if x[:1] == acronym[:1]
    ]:
        potential_match.append(text_token_list[potential_start_idx])
        idx = potential_start_idx + 1
        matches = ""
        for i, letter in enumerate(acronym[1:]):
            has_match = False
            stop_word_counter = 0
           
            if text_token_list[idx][:1] == letter:
                potential_match.append(orig_text_token_list[idx])
                matches = matches + letter
                idx = idx + 1
                has_match = True
               
            while stop_word_counter <= 2 and text_token_list[idx] in stop_w:
                potential_match.append(orig_text_token_list[idx])
                stop_word_counter = stop_word_counter + 1
                idx = idx + 1

            if text_token_list[idx][:1] == letter and has_match == False:
                potential_match.append(orig_text_token_list[idx])
                matches = matches + letter
                idx = idx + 1
                has_match = True

        if matches != acronym[1:]:
            potential_match = []
        if matches == acronym[1:]:
            break
    try:
        if potential_match[-1].title() in stop_w:
            potential_match = potential_match[:-1]
        result = " ".join(potential_match)
    except:
        result = None
    return result

def words_in_string(word_list, a_string):
    return set(word_list).intersection(a_string.split())

#Driver Code
s = read_file("acronym_example1.txt")
acronyms = find_acronyms(s)
d = find_long_forms(s, acronyms)
new_s = replace_acronyms(s, d)

In [2]:
find_acronyms(s)

['MRI', 'fMRI', 'BOLD', 'DMN', 'RSN', 'EEG', 'NIRS']

In [4]:
find_long_forms(s, acronyms)

{'MRI': None,
 'fMRI': None,
 'BOLD': 'Blood oxygen level dependent',
 'DMN': 'Default mode network',
 'RSN': 'Resting State Network',
 'EEG': None,
 'NIRS': None}